In [51]:
import requests
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime, timedelta
from skyfield.api import load, wgs84
from ics import Calendar, Event
import os
from zoneinfo import ZoneInfo


In [ ]:

# === CONFIGURATION ===
latitude, longitude = 44.4949, 11.3426  # Bologna
calendar_filename = "satellite_passes.ics"
json_filename = "satellite_passes.json"
csv_filename = "satellite_passes.csv"
natural_csv_filename = "satellite_passes_natural.csv"
local_tz = ZoneInfo("Europe/Rome")
maxdays = 60

In [53]:

# === SATELLITES TO TRACK ===
satellite_names = ["LANDSAT 8", "LANDSAT 9", "SENTINEL-2A", "SENTINEL-2B", "TERRA", "AQUA"]

In [54]:
# === Load TLEs ===
tle_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
satellites = load.tle_file(tle_url)
sat_dict = {sat.name.upper(): sat for sat in satellites}

In [55]:
# === Function to find the last overpass ===
def get_last_overpass(sat, name, lat, lon, days_back=10, tz=ZoneInfo("Europe/Rome")):
    location = wgs84.latlon(lat, lon)
    now_local = datetime.now(tz)
    ts = load.timescale()
    t1 = ts.from_datetime(now_local.astimezone(ZoneInfo("UTC")))
    t0 = ts.from_datetime((now_local - timedelta(days=days_back)).astimezone(ZoneInfo("UTC")))

    t, events = sat.find_events(location, t0, t1, altitude_degrees=10.0)
    passes = []
    current = {}
    for ti, event in zip(t, events):
        time_utc = ti.utc_datetime().replace(tzinfo=ZoneInfo("UTC"))
        time_local = time_utc.astimezone(tz)
        if event == 0:
            current["start"] = time_local
        elif event == 1:
            current["peak"] = time_local
        elif event == 2:
            current["end"] = time_local
            current["satellite"] = name
            if all(k in current for k in ("start", "peak", "end", "satellite")):
                passes.append(current)
            current = {}
    return passes[-1] if passes else None

In [56]:
# === Function to find future passes ===
def find_passes(sat, name, t0, t1, location, tz):
    t, events = sat.find_events(location, t0, t1, altitude_degrees=10.0)
    passes = []
    current = {}
    for ti, event in zip(t, events):
        time_utc = ti.utc_datetime().replace(tzinfo=ZoneInfo("UTC"))
        time_local = time_utc.astimezone(tz)
        if event == 0:
            current["start"] = time_local
        elif event == 1:
            current["peak"] = time_local
        elif event == 2:
            current["end"] = time_local
            current["satellite"] = name
            if all(k in current for k in ("start", "peak", "end", "satellite")):
                passes.append(current)
            current = {}
    return passes

In [57]:
# === Compute passes for all satellites ===
all_future_passes = []
location = wgs84.latlon(latitude, longitude)
ts = load.timescale()

for name in satellite_names:
    sat = sat_dict.get(name)
    if not sat:
        print(f"⚠️ Satellite {name} not found in TLE set.")
        continue

    last_pass = get_last_overpass(sat, name, latitude, longitude, days_back=maxdays, tz=local_tz)
    if not last_pass:
        print(f"⚠️ No recent pass for {name}")
        continue

    print(f"📡 Last pass for {name}: {last_pass['start'].strftime('%Y-%m-%d %H:%M')}")

    start_local = last_pass["start"]
    end_local = start_local + timedelta(days=60)

    t0 = ts.from_datetime(start_local.astimezone(ZoneInfo("UTC")))
    t1 = ts.from_datetime(end_local.astimezone(ZoneInfo("UTC")))

    future = find_passes(sat, name, t0, t1, location, local_tz)
    all_future_passes.extend(future)

📡 Last pass for LANDSAT 8: 2025-07-11 23:51
📡 Last pass for LANDSAT 9: 2025-07-11 23:01
📡 Last pass for SENTINEL-2A: 2025-07-12 00:35
📡 Last pass for SENTINEL-2B: 2025-07-12 10:55
📡 Last pass for TERRA: 2025-07-12 09:37
📡 Last pass for AQUA: 2025-07-12 05:23


In [58]:

# === Sort and export ===
all_future_passes.sort(key=lambda x: x["start"])
print(f"📅 Creating calendar file: {calendar_filename}")
cal = Calendar()
for p in all_future_passes:
    e = Event()
    e.name = f"{p['satellite']} Overpass - Bologna"
    e.begin = p["start"]
    e.end = p["end"]
    e.description = (
        f"{p['satellite']} will pass over Bologna\n"
        f"Start: {p['start']}\n"
        f"Peak: {p['peak']}\n"
        f"End: {p['end']}"
    )
    cal.events.add(e)

with open(calendar_filename, "w") as f:
    f.writelines(cal)
print(f"✅ Calendar saved to {calendar_filename}")


📅 Creating calendar file: satellite_passes_bologna.ics
✅ Calendar saved to satellite_passes_bologna.ics


In [ ]:
# === Save to JSON ===
print(f"💾 Saving data to JSON file: {json_filename}")
json_data = [
    {
        "satellite": p["satellite"],
        "start": p["start"].isoformat(),
        "peak": p["peak"].isoformat(),
        "end": p["end"].isoformat(),
    }
    for p in all_future_passes
]

with open(json_filename, "w") as f:
    json.dump(json_data, f, indent=2)

print(f"✅ JSON saved to {json_filename}")

In [63]:
# === Write natural language summary ===
def describe_passes(passes):
    lines = []
    for p in passes[:len(passes)]:
        start = p["start"].strftime("%A, %B %d %Y at %H:%M")
        peak = p["peak"].strftime("%H:%M")
        end = p["end"].strftime("%H:%M")
        lines.append(
            f"On {start}, {p['satellite']} will pass over Bologna. "
            f"It will reach peak visibility at {peak} and disappear below the horizon at {end}."
        )
    return lines

print("\n📝 Natural language summary of the next passes:")
for line in describe_passes(all_future_passes):
    print("- " + line) 


📝 Natural language summary of the next passes:
- On Saturday, July 12 2025 at 10:59, LANDSAT 9 will pass over Bologna. It will reach peak visibility at 11:03 and disappear below the horizon at 11:07.
- On Saturday, July 12 2025 at 11:04, SENTINEL-2A will pass over Bologna. It will reach peak visibility at 11:08 and disappear below the horizon at 11:12.
- On Saturday, July 12 2025 at 11:12, TERRA will pass over Bologna. It will reach peak visibility at 11:17 and disappear below the horizon at 11:22.
- On Saturday, July 12 2025 at 11:47, LANDSAT 8 will pass over Bologna. It will reach peak visibility at 11:52 and disappear below the horizon at 11:57.
- On Saturday, July 12 2025 at 12:33, SENTINEL-2B will pass over Bologna. It will reach peak visibility at 12:38 and disappear below the horizon at 12:43.
- On Saturday, July 12 2025 at 12:36, LANDSAT 9 will pass over Bologna. It will reach peak visibility at 12:41 and disappear below the horizon at 12:45.
- On Saturday, July 12 2025 at 12: